Resources:
- [Notes from Yanai](https://github.com/PastelBelem8/projects-notes/blob/master/research/2022-11-28/ELASTIC.md)

In [30]:
from elasticsearch import Elasticsearch

# Create an elastic search engine
es = Elasticsearch(
     cloud_id="lm-datasets:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJDk1N2U5ODIwZDUxNTQ0YWViMjk0MmQwNzI1NjE0OTQ2JDhkN2M0OWMyZDEzMTRiNmM4NDNhNGEwN2U4NDE5NjRl",
     api_key="T2w5Vl9ZTUIzVzE5dTlBblUtRlo6MHNBYWxhbHVTeGFodUpUOWIybkNNZw==",
     retry_on_timeout=True,
     http_compress=True,
)

Find out which indices exist:

In [11]:
indices = [name for name in es.indices.get(index="*").keys() if not name.startswith(".")]
indices

['c4',
 'laion1b-nolang',
 'laion2b-multi-1',
 'laion2b-multi-2',
 'metrics-endpoint.metadata_current_default',
 'openwebtext',
 're_laion2b-en-1',
 're_laion2b-en-2',
 're_oscar',
 're_pile',
 's2orc-abstracts',
 'search-test']

get mapping / schema from index:

In [12]:
es.indices.get_mapping(index="re_pile")

ObjectApiResponse({'re_pile': {'mappings': {'dynamic': 'false', 'properties': {'date': {'type': 'date'}, 'text': {'type': 'text'}, 'url': {'type': 'text'}}}}})

search over one index: 

In [13]:
query = { "match": { "text": "terrorist"} }
es.count(index="re_pile", query=query)

ObjectApiResponse({'count': 1224340, '_shards': {'total': 20, 'successful': 20, 'skipped': 0, 'failed': 0}})

In [14]:
results = es.search(index="re_pile", query=query)
results["hits"]["hits"][0]["_id"]

'train/23.jsonl.gz-6644456'

In [97]:
es.explain(index="re_pile", id="train/23.jsonl.gz-6644456", query=query)

ObjectApiResponse({'_index': 're_pile', '_id': 'train/23.jsonl.gz-6644456', 'matched': True, 'explanation': {'value': 11.080673, 'description': 'weight(text:terrorist in 997772) [PerFieldSimilarity], result of:', 'details': [{'value': 11.080673, 'description': 'score(freq=130.0), computed as boost * idf * tf from:', 'details': [{'value': 2.2, 'description': 'boost', 'details': []}, {'value': 5.150321, 'description': 'idf, computed as log(1 + (N - n + 0.5) / (n + 0.5)) from:', 'details': [{'value': 63315, 'description': 'n, number of documents containing term', 'details': []}, {'value': 10921091, 'description': 'N, total number of documents with field', 'details': []}]}, {'value': 0.97793305, 'description': 'tf, computed as freq / (freq + k1 * (1 - b + b * dl / avgdl)) from:', 'details': [{'value': 130.0, 'description': 'freq, occurrences of term within document', 'details': []}, {'value': 1.2, 'description': 'k1, term saturation parameter', 'details': []}, {'value': 0.75, 'description'

In [89]:
results["count"]

1224340

````python
results = {
    "took": int,
    "\_shards": {"total": int, "successful": int, "skipped": int, "failed": int}, # how many of the shards were successfully read or not.  
    "timed_out": bool,    # whether the search has timed out or not.
    "hits": hits, 
}
````
, where 

```python
hits = {
   "total": {
       "value": int,         # total value of hits (number of documents that match the query), 
                             # maximum by default is 10k
       "relation": str,      # it reflects how the specified value matches the true hits in the data
                             # (e.g., if it is "eq" then the number of hits is exactly equal to the 
                             # specified total.value)
   },
   "max_score": float,       # elastic search looks up for sentences ranking them by score.
   "hits": array<docs>[10],  # why is it only returning 10?
}
```

, where
```python
docs = {
    "_index": str,     # name of the search index
    "_id": str,        # name of the shard (?)
    "_score": float,   # score of the match of the search 
    "_source": {       # should contain the fields that describe a document.
        "text": str,   # the text in the document
        "meta": {      
            "pile_set_name": str, # subset of the pile dataset the text belongs to
        }
    },
}
```


In [78]:
len(results["hits"]["hits"])

10

In [79]:
results["took"], results["timed_out"], results["_shards"]

(67, False, {'total': 20, 'successful': 20, 'skipped': 0, 'failed': 0})

In [80]:
results["hits"].keys()

dict_keys(['total', 'max_score', 'hits'])

In [81]:
len(results["hits"]["hits"])

10

In [82]:
results["hits"]["hits"][0].keys()

dict_keys(['_index', '_id', '_score', '_source'])

In [83]:
results["hits"]["hits"][0]["_source"].keys()

dict_keys(['text', 'meta'])

In [84]:
print(results["hits"]["hits"][9]["_source"]["meta"])

{'pile_set_name': 'Wikipedia (en)'}


In [86]:
print(results["hits"]["hits"][0]["_source"]["text"])

63 Drills Since 93: Where the Drill and Reality Were ‘Coincidently’ Identical

That we Know of.*

Again, if you could stomach the propaganda of at least after the fact accessories to murder. You heard from all the ‘Talking Heads’ about what a great job they were doing with the recent ‘fertilizer explosion’ near Waco. Surely, you heard of the ‘coincidental’ drill?

“Originally there was a post describing the drill, saying that it would be complete with actors, who would take almost 200 local hospital beds. The original announcement read:

North Hills Hospital is proud to be hosting one of the largest emergency preparedness drills ever held in the state of Texas this week. We will be partnering with the North Central Texas Trauma Regional Advisory Council (NCTTRAC) and first responders from throughout North Texas to test our equipment and processes so that when a real disaster happens, we’re all ready to respond. (Which as we all know. this disaster has a miraculous chance of happening d

In [2]:
query = {
    "match": {
        "text": {
            "query": "hospital emergency room",
            "operator": "and",
        }
    }
}

results = es.search(index="re_pile", query=query, track_total_hits=True )
print(results["hits"]["total"])
results["hits"]["hits"]

{'value': 482226, 'relation': 'eq'}


[{'_index': 're_pile',
  '_id': 'train/13.jsonl.gz-3979528',
  '_score': 21.83589,
  '_source': {'text': '\n160 Ill. App.3d 173 (1987)\n513 N.E.2d 149\nDEBRA K. JOHNSON, Plaintiff-Appellant,\nv.\nRONALD E. SUMNER et al., Defendants (The Methodist Medical Center of Illinois, Defendant-Appellee).\nNo. 3-86-0713.\nIllinois Appellate Court \x97 Third District.\nOpinion filed September 3, 1987.\nJack Vieley, of Peoria (Dean R. Essig, of counsel), for appellant.\nWayne L. Hanold, of Westervelt, Johnson, Nicoll & Keller, of Peoria (James R. Morrison, of counsel), for appellee.\nJudgment affirmed.\nJUSTICE SCOTT delivered the opinion of the court:\nPlaintiff, Debra K. Johnson, brought this action for medical malpractice alleging that Drs. Ronald E. Sumner, William Hart, Lorris M. Bowers, and The Methodist Medical Center of Illinois were negligent when rendering her treatment in the emergency room and following her admission to the hospital. Johnson appeals the order entered by the circuit cour

In [8]:
query = {
    "match": {
        "text": {
            "query": "hospital emergency room",
            "operator": "and",
        }
    }
}

results = es.search(index="re_pile", query=query, size=50, track_total_hits=True )
print(results["hits"]["total"])
len(results["hits"]["hits"])

{'value': 482226, 'relation': 'eq'}


50

## Queries - explained

### Match queries

Match queries will look for the `<name_of_the_field_to_search_for>` and look for documents whose at least one of the terms in `<search_terms>` occurs. In other words, elastic search implicitly treat the terms in `<search_terms>` with an `OR` operator.

```python
query = {
    "match": { 
        <name_of_the_field_to_search_for>: {
            "query": <search_terms>,  # treated as a bag of words, no order implied
        }
    }
}

es.search(index=<index>, query=query)
```

To restrict matching to documents that have all the terms in the query:


```python
query = {
    "match": { 
        <name_of_the_field_to_search_for>: {
            "query": <search_terms>,  # treated as a bag of words, no order implied
            "operator": "and",
        }
    }
}


es.search(index=<index>, query=query)
```

You'll likely get fewer options, and sometimes the `AND` operator is too restrictive. In those cases, you can use a different approach: specify the minimum number of tokens in the `<search_terms>` that you'd like the documents to satisfy. Here is an example, whose returned documents would have to match at least 3 out of the terms you specified in `<search_terms>`:



```python
query = {
    "match": { 
        <name_of_the_field_to_search_for>: {
            "query": <search_terms>,  # treated as a bag of words, no order implied
            "minimum_should_match": 3,
        }
    }
}

es.search(index=<index>, query=query)
```

## "match_phrase" query

Instead of treating `<search_terms>` as a bag of words, it treats them as a phrase (i.e., contiguous sequence of words), all retrieved documents will satisfy:
1. All terms appear in the specified field.
2. Terms appear in the specified order.
3. Terms appear next to each other.


```python
query = {
    "match_phrase": { 
        <name_of_the_field_to_search_for>: {
            "query": <search_phrase>
        }
    }
}

es.search(index=<index>, query=query)
```


## "multi_match" query

Sometimes, we might want to match documents against multiple queries. This will look for documents whose `<search_terms>` are at least in one of the specified fields. 

```python
query = {
    "multi_match": { 
        "query": <search_phrase>,
        "fields": [
            "<field_1>", 
            ...
            "<field_n>",
        ]
    }
}

es.search(index=<index>, query=query)
```

By default, ElasticSearch will give the same value to all the fields. To boost a specific field, you can use the suffix `^<num>`. It will return the same hits but the ranking is changed.


```python
query = {
    "multi_match": { 
        "query": <search_phrase>,
        "fields": [
            "<field_1>^2",  # added the suffix ^2
            ...
            "<field_n>",
        ]
    }
}

es.search(index=<index>, query=query)
```

**note**: by default, multi_match performs a **match** query on multiple fields, which means it is matching one of the words specified query, regardless of order.

To improve this, we can specify a `"type"` parameter and set it to **phrase**.

```python
query = {
    "multi_match": { 
        "query": <search_phrase>,
        "fields": [
            "<field_1>^2",  # added the suffix ^2
            ...
            "<field_n>",
        ],
        "type": "phrase"
    }
}

es.search(index=<index>, query=query)
```

## Combined queries

When we have multiple queries we want to be looked for. In that case, we have to use a **bool query**.


These enable us to sort and filter the returned queries as our preference. The general syntax is: 

```python
query = {
    "bool": { 
        "must": [{<one_or_more_queries_should_be_specified_here>}],     # a document must match all 
                                                                        # these queries. 
        "must_not": [{<one_or_more_queries_should_be_specified_here>}], # a document MUST NOT match
                                                                        # any of these queries.
        "should": [{<one_or_more_queries_should_be_specified_here>}],   # a documents does not have
                                                                        # to match any of these queries, 
                                                                        # but will be ranked higher if it does  
        "filter": [{<one_or_more_queries_should_be_specified_here>}],   # filter documents 
                                                                        # based on the specified filters
                                                                        # (only the ones satisfying these filters
                                                                        # are considered for matching)
    }
}

es.search(index=<index>, query=query)
```

The `filter` option will filter out documents based on the specified conditions.

One example that will filter documents that mention "Michele Obama" in the "headline" field and also match "POLITICS" in the "category" field: 

```python
query = {
    "bool": { 
        "must": [
            {"match_phrase": {"headline": "Michele Obama"}},
            {"match": {"category": "Politics"}},
        ], 
    }
}

es.search(index=<index>, query=query)
```

Note the previous example will not work in the AI2 index, since we only have two fields per document, named "text" and "meta".



### filter query

Let us filter some documents by some time range. It will return all documents that match the phrase "Michele Obama" and are categorized as "Politics" and occur between "2014/03/25 - 2016/03/25".

```python
query = {
    "bool": { 
        "must": [
            {"match_phrase": {"headline": "Michele Obama"}},
            {"match": {"category": "Politics"}},
        ], 
        
        "filter": {
            "range": {
                "date": { # date field if it exists (in PILE it does not)
                    "gte": "2014-03-25",
                    "lte": "2016-03-25",
                }
            }
        }
    }
}

es.search(index=<index>, query=query)
```

## Aggregation queries

(Todo: [watch the video](https://www.youtube.com/watch?v=iGKOdep1Iss&list=PL_mJOmq4zsHZYAyK606y7wjQtC0aoE6Es&index=5))


```python
query = {
    "aggregations": {
        <name_of_resulting_aggregation>: {
            "terms": {
                "field": <name_of_the_field_to_agg_by>,
                "size": <batch_size_of_categories>,
            }
        }
    }
}
```

The following query returns the set of "<name_of_the_field_to_agg_by>" whose documents contain the specified terms.

```python
query = {
    "match": {
        "field": <search_terms>
    },
    "aggregations": {
        <name_of_resulting_aggregation>: {
            "terms": {
                "field": <name_of_the_field_to_agg_by>,
                "size": <batch_size_of_categories>,
            }
        }
    }
}
```


## Iterating the set of documents

In [29]:
def search_all_docs(es, index, query, size=250, **kwargs):
    # SIZE is the number of hits to return with each query
    
    result = es.search(index=index, query=query, track_total_hits=True, size=size, **kwargs)
    print("Reported docs in ES:", result["hits"]["total"])

    docs = result["hits"]["hits"]
    
    count = len(docs)
    offset = len(docs)
    
    while len(docs) > 0:
        result = es.search(index=index, query=query, from_=offset, size=size, **kwargs)
        docs = result["hits"]["hits"]
        
        count += len(docs)
        offset += len(docs)
        
    print("Total docs", count)
    
    
search_all_docs(es, "re_pile", query)

Reported docs in ES: {'value': 291, 'relation': 'eq'}
Total docs 291


The example above, will likely reuslt in a error. The max_result_window parameter is set to avoid jeopardizing hte performance of the nodes. From the elasticsearch.search documentation: 

> Starting document offset. By default, you cannot page through more than 10,000 hits using the from and size parameters. To page through more hits, use the search_after parameter.


To do this type of query, the documentation suggests using the **search_after** parameter (, [Elastic search documentation](https://www.elastic.co/guide/en/elasticsearch/reference/current/paginate-search-results.html#search-after)).

In [32]:
def search_all_docs(es, index, query, size=250, **kwargs):
    # SIZE is the number of hits to return with each query
    
    result = es.search(index=index, query=query, track_total_hits=True, size=size, **kwargs)
    print("Reported docs in ES:", result["hits"]["total"])

    docs = result["hits"]["hits"]
    
    count = len(docs)
    offset = len(docs)
    
    while len(docs) > 0:
        result = es.search(index=index, query=query, from_=offset, size=size, **kwargs)
        docs = result["hits"]["hits"]
        
        count += len(docs)
        offset += len(docs)
        
    print("Total docs", count)
    
    
search_all_docs(es, "re_pile", query)

Reported docs in ES: {'value': 291, 'relation': 'eq'}
Total docs 291


In [19]:
results["hits"]["hits"][0]["_id"]

'train/23.jsonl.gz-6644456'

In [26]:
query = {
    "match": {
        "text": {
            "query": "Catarina Belem",
            "operator": "and",
        }
    }
}

sort = [
    {"_id": "asc"}
]


results = es.search(index="re_pile", query=query, sort=sort, track_total_hits=True )
print(results["hits"]["total"])
len(results["hits"]["hits"])

BadRequestError: BadRequestError(400, 'search_phase_execution_exception', 'Fielddata access on the _id field is disallowed, you can re-enable it by updating the dynamic cluster setting: indices.id_field_data.enabled')

Since the field `_id` is private, we cannot use it (it doesn't effectively belong to the fields made available for each document). Therefore we have to find alternatives:

In [ ]:

def to_json(data, filepath, mode="a"):
    import json
    data = data if isinstance(data, (list, tuple)) else [data]
    data = [json.dumps(l, sort_keys=True) for l in data]
    
    with open(filepath, mode=mode) as f:
        f.writelines(data)
        
# Sanity check
docs_iter = iter(scroll(query, 100, "10m"))
to_json(next(docs_iter), "temp.json", "w")
to_json(next(docs_iter), "temp.json")